### SPLIT AFRICA DATASET 192

I re-ran this notebook until the split was reasonably even between train/validation/test. In a perfect split the `low-med-high rfracs` would all equal 1 and the `block split fracs` would equal the respective `group fracs`.  I am not too concerned with less validation data, but prefer that `water-tag=low` be well represented in both test and validation.
 

---

In [1]:
import pandas as pd
from random import sample

In [2]:
WSIZE=192
DSET=f'surface-water.africa.win{WSIZE}'
NO_DATA_MAX=0.05
IDENTS=['group_id','window_index']

In [3]:
df=pd.read_csv(f'{DSET}.csv')
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=IDENTS).shape[0])
df=df[df.no_data<=NO_DATA_MAX]
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=IDENTS).shape[0])
df.sample(3)

8708 632 8541
7527 600 7385


,gsw_path,s1_path,block_id,group_id,window_index,window,water,not_water,no_data,dataset
864,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_21_-34,group_21.92_-34.24,3,"(192, 192, 192, 192)",0.059706,0.929253,0.011041,africa
2307,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_-14_10,group_-14.58_10.85,0,"(0, 0, 192, 192)",0.011963,0.988037,0.000000,africa
4545,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_22_-34,group_22.13_-34.07,0,"(0, 0, 192, 192)",0.049181,0.947076,0.003743,africa


---

In [4]:
def water_counter(df,quantiles=[0.15,0.85],labels=['low','medium','high']):
    wqs=[0]+[df.water.quantile(q) for q in quantiles]+[1+1e-8]
    dfs=[]
    for i,l in enumerate(labels):
        _df=df.copy()[(df.water>=wqs[i])&(df.water<wqs[i+1])]
        _df['water_tag']=l
        dfs.append(_df)
    return pd.concat(dfs).sample(frac=1)


def split_groups(blocks,valid_frac=0.2,test_frac=0.1,hold_out_frac=0.2,relative_frac=True):
    if relative_frac:
        valid_frac=(1-hold_out_frac)*valid_frac
        test_frac=(1-hold_out_frac)*test_frac
    nb_blocks=len(blocks)
    nb_valid=int(valid_frac*nb_blocks)    
    nb_test=int(test_frac*nb_blocks)
    nb_hold_out=int(hold_out_frac*nb_blocks)
    blocks=sample(blocks,nb_blocks)
    valid=blocks[:nb_valid]
    test=blocks[nb_valid:nb_valid+nb_test]
    hold_out=blocks[nb_valid+nb_test:nb_valid+nb_test+nb_hold_out]
    train=blocks[nb_valid+nb_test+nb_hold_out:]
    rtotal=len(train)+len(valid)+len(test)
    print('block relative split:',len(train)/rtotal,len(valid)/rtotal,len(test)/rtotal,len(hold_out)/nb_blocks)
    print('block split fracs:',len(train)/nb_blocks,len(valid)/nb_blocks,len(test)/nb_blocks,len(hold_out)/nb_blocks)
    return train, valid, test, hold_out


def sub_df(typ,df,blocks,cnt):
    
    sdf=df.copy()[df.block_id.isin(blocks)]
    sdf['data_split']=typ
    
    sgcnt=sdf.drop_duplicates(subset=IDENTS).shape[0]
    frac=sgcnt/cnt
    
    ltcnt=df[df.water_tag=='low'].shape[0]*frac
    mtcnt=df[df.water_tag=='medium'].shape[0]*frac
    htcnt=df[df.water_tag=='high'].shape[0]*frac
    
    sltcnt=sdf[sdf.water_tag=='low'].shape[0]
    smtcnt=sdf[sdf.water_tag=='medium'].shape[0]
    shtcnt=sdf[sdf.water_tag=='high'].shape[0]
    print()
    print(f'{typ} group:',frac,f'({sgcnt}/{cnt})  [{int(ltcnt)},{int(mtcnt)},{int(htcnt)}]')
    print('* low-med-high rfracs:',sltcnt/ltcnt,smtcnt/mtcnt,shtcnt/htcnt)
    return sdf
    
    
def df_splitter(df,train, valid, test, hold_out):
    cnt=df.drop_duplicates(subset=IDENTS).shape[0]
    tdf=sub_df('train',df,train,cnt)    
    vdf=sub_df('valid',df,valid,cnt)    
    sdf=sub_df('test',df,test,cnt)    
    hdf=sub_df('hold_out',df,hold_out,cnt)
    df=pd.concat([tdf, vdf, sdf, hdf])
    return df

---

In [5]:
df=water_counter(df)

In [6]:
blocks=list(df.block_id.unique())

In [7]:
print('-'*85)
train, valid, test, hold_out=split_groups(blocks)
print('-'*85)
df=df_splitter(df, train, valid, test, hold_out)

-------------------------------------------------------------------------------------
block relative split: 0.7 0.2 0.1 0.2
block split fracs: 0.56 0.16 0.08 0.2
-------------------------------------------------------------------------------------

train group: 0.589844278943805 (4356/7385)  [665,3108,665]
* low-med-high rfracs: 0.9483810949599151 0.9847258595964823 1.1037126641241304

valid group: 0.12728503723764387 (940/7385)  [143,670,143]
* low-med-high rfracs: 1.009907386449374 1.0286345835520208 0.8628799728624467

test group: 0.0974949221394719 (720/7385)  [109,513,110]
* low-med-high rfracs: 1.209373768715524 1.031533312249631 0.6995435980710559

hold_out group: 0.1853757616790792 (1369/7385)  [209,976,209]
* low-med-high rfracs: 1.0473264379963634 1.012354756931402 0.9221687874166747


---

In [8]:
df.sample(3)

,gsw_path,s1_path,block_id,group_id,window_index,window,water,not_water,no_data,dataset,water_tag,data_split
2933,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_33_0,group_33.06_0.66,0,"(0, 0, 192, 192)",0.009766,0.989746,0.000488,africa,low,test
1435,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_-2_33,group_-2.91_33.62,3,"(192, 192, 192, 192)",0.014784,0.984972,0.000244,africa,medium,train
125,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_-15_11,group_-15.04_11.07,1,"(0, 192, 192, 192)",0.073975,0.926025,0.000000,africa,medium,train


In [9]:
file_name=f'{DSET}.split.csv'
df.to_csv(file_name,index=False)
print(f'UPLOAD: gsutil cp {file_name} gs://surface-water-public/data/v1/datasets')

UPLOAD: gsutil cp surface-water.africa.win192.split.csv gs://surface-water-public/data/v1/datasets


---

In [15]:
# !gsutil cp surface-water.africa.win192.split.csv gs://surface-water-public/data/v1/datasets

---

##### WATER/NOT RATIO

In [11]:
_df=df[df.data_split.isin(['train','valid'])]

In [12]:
_df.not_water.mean()/_df.water.mean()

7.917973905779916

In [13]:
_df.drop_duplicates(subset=IDENTS)[['water','not_water']].describe()

,water,not_water
count,5296.000000,5296.000000
mean,0.111384,0.884144
std,0.127563,0.128587
min,0.005018,0.373372
25%,0.023546,0.841756
50%,0.057766,0.937446
75%,0.152540,0.972460
max,0.599609,0.994982
